# **Thư viện**

In [20]:
import pandas as pd
import numpy as np
import keras_tuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


# **Dữ liệu**

In [21]:
data = pd.read_csv("../data/processed/EUR_VND_Exchange_Rate.csv")
data.head()

,Date,Transfer
0,2020-04-01,0.569770
1,2020-04-02,0.569770
2,2020-04-03,0.469007
3,2020-04-04,0.469007
4,2020-04-05,0.469007


In [22]:
timestamps = data["Date"].values  
values = data["Transfer"].fillna(0).astype(np.float32).values

train_size = int(len(values) * 0.8)
train_values, test_values = values[:train_size], values[train_size:]
train_timestamps, test_timestamps = timestamps[:train_size], timestamps[train_size:]


In [23]:
seq_length = 30
def create_sequences(values, timestamps):
        """Tạo sequences với timestamps tương ứng."""
        if len(values) <= seq_length:
            print("Dữ liệu quá ngắn! Không thể tạo sequences.")
            return np.array([]), np.array([]), []

        X, y, y_timestamps = [], [], []
        for i in range(len(values) - seq_length):
            X.append(values[i : i + seq_length])
            y.append(values[i + seq_length])
            y_timestamps.append(pd.to_datetime(timestamps[i + seq_length]).date())

        return np.array(X, dtype=np.float32), np.array(y, dtype=np.float32), np.array(y_timestamps)


X_train, y_train, time_train = create_sequences(train_values, train_timestamps)
X_test, y_test, time_test = create_sequences(test_values, test_timestamps)

In [24]:
X_train.shape

(1431, 30)

In [25]:
X_test.shape

(336, 30)

# **Mô hình LSTM**

In [ ]:
def build_lstm_model(hp, seq_length):
    """Xây dựng mô hình LSTM với hyperparameter tuning."""
    model = Sequential()
    model.add(Input(shape=(seq_length, 1)))

    model.add(LSTM(
        units=hp.Int("lstm_units_2", 32, 128, 16),
        return_sequences=False,
        activation="tanh",
        kernel_regularizer=l2(hp.Choice("l2_reg", [0.001, 0.01, 0.1]))
    ))
    model.add(BatchNormalization())
    model.add(Dropout(hp.Float("dropout_2", 0.1, 0.5, 0.1)))

    model.add(Dense(hp.Int("dense_units", 16, 64, 16), activation="relu"))
    model.add(Dense(1))

    model.compile(
        optimizer=Adam(learning_rate=hp.Choice("learning_rate", [0.001, 0.0005, 0.0001])),
        loss="mse",
        metrics=["mae"]
    )

    return model

def train_lstm(X_train, y_train, X_test, y_test, seq_length, model_path, project_name):
    """Huấn luyện và lưu mô hình LSTM sử dụng hyperparameter tuning."""
    tuner = kt.RandomSearch(
        lambda hp: build_lstm_model(hp, seq_length),
        objective="val_loss",
        max_trials=10,
        executions_per_trial=2,
        directory=r"C:\Users\DELL\Downloads\eur-vnd-exchange-rate\models",
        project_name=project_name
    )

    early_stopping = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)
    model_checkpoint = ModelCheckpoint(model_path, save_best_only=True)

    tuner.search(X_train, y_train,
                 epochs=50,
                 batch_size=32,
                 validation_data=(X_test, y_test),
                 callbacks=[early_stopping, model_checkpoint])

    return tuner.get_best_hyperparameters(num_trials=1)[0]


In [28]:
best_hp = train_lstm(
    X_train, y_train, 
    X_test, y_test, 
    seq_length=30, 
    model_path=r"C:\Users\DELL\Downloads\eur-vnd-exchange-rate\models\LSTM\best_lstm.h5", 
    project_name="LSTM"
)


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
128               |128               |lstm_units_1
0.001             |0.001             |l2_reg
0.4               |0.4               |dropout_1
80                |80                |lstm_units_2
0.1               |0.1               |dropout_2
64                |64                |dense_units
0.0005            |0.0005            |learning_rate

Epoch 1/50
44/45 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.5491 - mae: 0.4200

45/45 ━━━━━━━━━━━━━━━━━━━━ 7s 43ms/step - loss: 0.5436 - mae: 0.4165 - val_loss: 0.6658 - val_mae: 0.6896
Epoch 2/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.3037 - mae: 0.2551

45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 0.3033 - mae: 0.2548 - val_loss: 0.5802 - val_mae: 0.6293
Epoch 3/50
44/45 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.2669 - mae: 0.2285

45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 0.2667 - mae: 0.2282 - val_loss: 0.4891 - val_mae: 0.5582
Epoch 4/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 0.2407 - mae: 0.2017

45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 50ms/step - loss: 0.2406 - mae: 0.2016 - val_loss: 0.4312 - val_mae: 0.5096
Epoch 5/50
44/45 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.2253 - mae: 0.1837

...

45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 62ms/step - loss: 0.0387 - mae: 0.0596 - val_loss: 0.0328 - val_mae: 0.0272
Epoch 36/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 48ms/step - loss: 0.0378 - mae: 0.0629 - val_loss: 0.0332 - val_mae: 0.0503
Epoch 37/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 46ms/step - loss: 0.0372 - mae: 0.0684 - val_loss: 0.0351 - val_mae: 0.0738
Epoch 38/50
43/45 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 0.0361 - mae: 0.0698

45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 43ms/step - loss: 0.0360 - mae: 0.0695 - val_loss: 0.0289 - val_mae: 0.0330
Epoch 39/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step - loss: 0.0348 - mae: 0.0702 - val_loss: 0.0298 - val_mae: 0.0550
Epoch 40/50
44/45 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 0.0330 - mae: 0.0648

45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 51ms/step - loss: 0.0330 - mae: 0.0648 - val_loss: 0.0258 - val_mae: 0.0240
Epoch 41/50
44/45 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 0.0320 - mae: 0.0688

45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 53ms/step - loss: 0.0320 - mae: 0.0687 - val_loss: 0.0246 - val_mae: 0.0226
Epoch 42/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 48ms/step - loss: 0.0306 - mae: 0.0659 - val_loss: 0.0316 - val_mae: 0.0873
Epoch 43/50
35/45 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 0.0310 - mae: 0.0712

KeyboardInterrupt: 

In [ ]:
best_hp.values

NameError: name 'best_hp' is not defined